In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


from index_travel_accessibility.travel_time_and_centroid import (
    get_hospital_df,
    
)

In [2]:
# Modifying the existing hospitals DataFrame to Prediction DataFrame format
existing_hospitals = get_hospital_df()

existing_hospitals.rename(columns={
    'MaxBeds': 'bed_allocation',
    'SiteID': 'node'
}, inplace=True)

In [3]:
# Adding Geometry column to existing_hospitals
existing_hospitals['Lat'] = existing_hospitals['Lat'].round(6)
existing_hospitals['Lon'] = existing_hospitals['Lon'].round(6)
existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)


In [4]:
# Getting district codes from the existing hospitals

# Load Saarland NUTS level 3 shapefile
nuts = gpd.read_file('./../index_travel_accessibility/data/raw/NUTS_RG_01M_2021_4326_LEVL_3.shp')
nuts = nuts.to_crs('EPSG:4326')


existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')


In [5]:
# Spatial join hospitals with NUTS polygons (only needed columns)
existing_hospitals = gpd.sjoin(
    existing_hospitals,
    nuts[['geometry', 'NUTS_NAME']],
    how='left',
    predicate='within'
)


In [6]:
# Map NUTS_NAME to your official Saarland district codes
name_to_code = {
    "Regionalverband Saarbrücken": "10041",
    "Merzig-Wadern": "10042",
    "Neunkirchen": "10043",
    "Saarlouis": "10044",
    "Saarpfalz-Kreis": "10045",
    "St. Wendel": "10046"
}

# Assign district_code based on district_name
existing_hospitals['district_code'] = existing_hospitals['NUTS_NAME'].map(name_to_code)

# Rename NUTS_NAME to district_name
existing_hospitals.rename(columns={'NUTS_NAME': 'district_name'}, inplace=True)

In [7]:
# Select and reorder columns as needed
existing_hospitals = existing_hospitals[['geometry', 'district_code', 'node', 'bed_allocation', 'Lat', 'Lon']]

existing_hospitals['type'] = 'existing'

In [8]:
existing_hospitals.to_excel("results/status_quo.xlsx", index=False)

In [9]:
from index_travel_accessibility.travel_time_and_centroid import (
    map_predicted_and_existing_hospitals,
)

map_predicted_and_existing_hospitals(
    'results/maps/status_quo.html', 
    existing_hospitals
)

✅ Map saved with existing and predicted hospitals!
